In [2]:
!pip install nest_asyncio



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Waleed\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from supabase.client import create_client
import openai
import os
from dotenv import load_dotenv
from fastapi import FastAPI
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

In [2]:
load_dotenv()

openai.api_key = os.environ['OPENAI_API_KEY']
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase = create_client(supabase_url, supabase_key)
embeddings = OpenAIEmbeddings()
vector_store = SupabaseVectorStore(
    embedding=embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)

c:\Users\Waleed\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
app = FastAPI()
class QueryRequest(BaseModel):
    query: str


In [4]:
client = openai

In [5]:
@app.post("/legal-query/")
async def handle_query(request: QueryRequest):
    query = request.query
   
    matched_docs = vector_store.similarity_search(query)
    injected_docs = "\n\n".join([doc.page_content for doc in matched_docs]) if matched_docs else ""
    
    if not injected_docs.strip():
        completion_messages = [
            {
                "role": "system",
                "content": """
                [Context Handling]
                When responding to queries, consider the following:
                 - If the current query is related to previous interactions, maintain context from those prior queries. Reference relevant details from previous interactions to provide a coherent response.
                 - For each query, analyze if it builds upon or is connected to previous queries. If so, use the historical context to inform your answer.
                
                [General Instructions]
                As an AI with comprehensive expertise in Pakistani law, your primary role is to provide accurate, detailed, and legally sound answers to any legal queries. When engaging with queries:
                - Comprehend: Fully understand each query with a focus on the legal context, relevant laws, and statutes in Pakistan.
                - Response: Provide accurate, clear, and concise legal answers using the relevant documents provided. Make sure to reference the specific law, act, or regulation from which the information is derived Please ensure that only queries related to legal matters, including specific laws, legal advice, and related topics, are addressed. Do not respond to queries that pertain to unrelated subjects such as the FIFA World Cup, best movies, health issues, or other non-legal topics.
                - Reference and Suggest: Support your answers with references to the relevant legal documents, ensuring information reliability and legal accuracy. Suggest additional laws or resources for further reading or insights if applicable. When referencing a decision of court, provide text from the decision along with case name, number and tagline for better reference and transparency.
                - Expertise Beyond Documents: If the information is not in the provided documents then display answer by querying and fetching data from openAI server or scrap from the internet.
                """
            },
            {
                "role": "user",
                "content": query
            }
        ]
    else:
        completion_messages = [
            {
                "role": "system",
                "content": """
                [Context Handling]
                When responding to queries, consider the following:
                 - If the current query is related to previous interactions, maintain context from those prior queries. Reference relevant details from previous interactions to provide a coherent response.
                 - For each query, analyze if it builds upon or is connected to previous queries. If so, use the historical context to inform your answer.
                
                [General Instructions]
                As an AI with comprehensive expertise in Pakistani law, your primary role is to provide accurate, detailed, and legally sound answers to any legal queries. When engaging with queries:
                - Comprehend: Fully understand each query with a focus on the legal context, relevant laws, and statutes in Pakistan.
                - Response: Provide accurate, clear, and concise legal answers using the relevant documents provided. Make sure to reference the specific law, act, or regulation from which the information is derived Please ensure that only queries related to legal matters, including specific laws, legal advice, and related topics, are addressed. Do not respond to queries that pertain to unrelated subjects such as the FIFA World Cup, best movies, health issues, or other non-legal topics.
                - Reference and Suggest: Support your answers with references to the relevant legal documents, ensuring information reliability and legal accuracy. Suggest additional laws or resources for further reading or insights if applicable. When referencing a decision of court, provide text from the decision along with case name, number and tagline for better reference and transparency
                - Expertise Beyond Documents: If the information is not in the provided documents then display answer by querying and fetching data from openAI server or scrap from the internet.
                """
            },
            {
                "role": "user",
                "content": query
            },
            {
                "role": "assistant",
                "content": injected_docs
            }
        ]
        response = client.chat.completions.create(
        model="gpt-4o",
        messages=completion_messages,
        max_tokens=1024,
        temperature=0.2
    )
        return response


In [6]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [14852]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
2024-08-20 13:10:14,278:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-20 13:10:15,463:INFO - HTTP Request: POST https://gewsevbxofuwqzufabvq.supabase.co/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"
2024-08-20 13:10:29,551:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:     127.0.0.1:51104 - "POST /legal-query/ HTTP/1.1" 200 OK


2024-08-20 13:12:07,139:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-20 13:12:08,032:INFO - HTTP Request: POST https://gewsevbxofuwqzufabvq.supabase.co/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"
2024-08-20 13:12:33,461:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:     127.0.0.1:51127 - "POST /legal-query/ HTTP/1.1" 200 OK


2024-08-20 13:15:25,017:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-20 13:15:25,919:INFO - HTTP Request: POST https://gewsevbxofuwqzufabvq.supabase.co/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"
2024-08-20 13:15:43,353:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:     127.0.0.1:51166 - "POST /legal-query/ HTTP/1.1" 200 OK


2024-08-20 13:19:21,828:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-20 13:19:22,770:INFO - HTTP Request: POST https://gewsevbxofuwqzufabvq.supabase.co/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"
2024-08-20 13:19:37,130:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:     127.0.0.1:51238 - "POST /legal-query/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14852]


KeyboardInterrupt: 

In [7]:
query = "Are ASF employers bound under army act 1952"
matched_docs = vector_store.similarity_search(query)
injected_docs = "\n\n".join([doc.page_content for doc in matched_docs])

2024-08-20 01:33:49,390:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-20 01:33:50,447:INFO - HTTP Request: POST https://gewsevbxofuwqzufabvq.supabase.co/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"


NameError: name 'completion_messages' is not defined

In [8]:
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [10360]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10360]


KeyboardInterrupt: 